In [ ]:
import cv2
import numpy as np

# Load the main image
im = cv2.imread('ss.jpg')
imG = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

# Load the template image
imT = cv2.imread('object1.png', 0)

# Perform template matching
result = cv2.matchTemplate(imG, imT, cv2.TM_CCOEFF_NORMED)

# Define a threshold to determine if a match is found
threshold = 0.8

# Get the locations where the template matches the main image above the threshold
locations = np.where(result >= threshold)

# Draw rectangles around the matching regions
for loc in zip(*locations[::-1]):
    pt1 = loc
    pt2 = (pt1[0] + imT.shape[1], pt1[1] + imT.shape[0])
    cv2.rectangle(im, pt1, pt2, (0, 255, 0), 2)

# Display the result
imS = cv2.resize(im, (960, 540)) 
cv2.imshow('Result', imS)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# Import essential libraries
import requests
import cv2
import numpy as np
import imutils

# Replace the below URL with your own. Make sure to add "/shot.jpg" at last.
url = "http://192.168.50.15:8080/shot.jpg"

# Load the template image
imT = cv2.imread('object1.png', 0)

# Define a threshold to determine if a match is found
threshold = 0.7

# While loop to continuously fetching data from the Url
while True:
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    im = cv2.imdecode(img_arr, -1)
    #im = imutils.resize(img, width=1000, height=1800)

    # Load the main image
    imG = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    # Perform template matching
    result = cv2.matchTemplate(imG, imT, cv2.TM_CCOEFF_NORMED)

    # Get the locations where the template matches the main image above the threshold
    locations = np.where(result >= threshold)

    # Draw rectangles around the matching regions
    for loc in zip(*locations[::-1]):
        pt1 = loc
        pt2 = (pt1[0] + imT.shape[1], pt1[1] + imT.shape[0])
        cv2.rectangle(im, pt1, pt2, (0, 255, 0), 2)

    imS = cv2.resize(im, (960, 540)) 
    cv2.imshow("Android_cam", imS)

    # Press Esc key to exit
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np

# Load the main image
main_image = cv2.imread('ss.jpg', 0)

# Load the template image
template_image = cv2.imread('object1.png', 0)

# Initialize ORB detector
orb = cv2.ORB_create()

# Detect keypoints and compute descriptors for the template image
template_keypoints, template_descriptors = orb.detectAndCompute(template_image, None)

# Initialize Brute-Force matcher
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Detect keypoints and compute descriptors for the resized image
main_keypoints, main_descriptors = orb.detectAndCompute(main_image, None)

# Match descriptors between template and resized image
matches = bf.match(template_descriptors, main_descriptors)

# Sort the matches by distance
matches = sorted(matches, key=lambda x: x.distance)

# Get the top N matches (adjust N as needed)
N = 10
top_matches = matches[:N]

# Draw matches on the resized image
matched_image = cv2.drawMatches(template_image, template_keypoints,
                                main_image, main_keypoints, top_matches, None,
                                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Display the matched image
cv2.imshow('Matched Image', matched_image)
cv2.waitKey(0)

cv2.destroyAllWindows()


In [ ]:
# Import essential libraries
import requests
import cv2
import numpy as np
import imutils
from time import sleep

# Replace the below URL with your own. Make sure to add "/shot.jpg" at last.
url = "http://192.168.50.15:8080/shot.jpg"

# Load the template image
template_image_paths = ['object1.png','object2.png']
found_templates  = [ '' for i in range(len(template_image_paths)) ]
# Define a threshold to determine if a match is found
threshold = 0.7

# While loop to continuously fetching data from the Url
while True:
    sleep(1)
    
    for i, template_image_path in enumerate(template_image_paths):
        
        img_resp = requests.get(url)
        img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
        main_image = cv2.imdecode(img_arr, -1)
        #im = imutils.resize(img, width=1000, height=1800)

        template_image = cv2.imread(template_image_path, 0)
        
        # Initialize ORB detector
        orb = cv2.ORB_create()

        # Detect keypoints and compute descriptors for the template image
        template_keypoints, template_descriptors = orb.detectAndCompute(template_image, None)

        # Initialize Brute-Force matcher
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

        # Detect keypoints and compute descriptors for the resized image
        main_keypoints, main_descriptors = orb.detectAndCompute(main_image, None)

        # Match descriptors between template and resized image
        matches = bf.match(template_descriptors, main_descriptors)

        # Sort the matches by distance
        matches = sorted(matches, key=lambda x: x.distance)

        # Set a distance threshold to filter good matches
        distance_threshold = 100

        # Store the matching keypoints from the main image
        matching_keypoints = []

        # Iterate over matches and filter based on distance threshold
        for match in matches:
            if match.distance < distance_threshold:
                matching_keypoints.append(main_keypoints[match.trainIdx])
        
        if len(matching_keypoints) >= 15:
            found_templates[i] = template_image_path
        else:
            found_templates[i] = ''
        
        # Draw matches on the resized image
        matched_image = cv2.drawMatches(template_image, template_keypoints,
                                    main_image, main_keypoints, top_matches, None,
                                    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        matched_image_scaled = cv2.resize(matched_image, (960, 540)) 
        cv2.imshow("Android_cam" + template_image_path, matched_image_scaled)
        
    
    
    print(found_templates)
    # Press Esc key to exit
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
